In [ ]:
#Installing required libraries
!pip install transformers datasets torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Import Necessary Libraries

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Load Dataset

In [ ]:
# Loading the OpenWebText dataset
dataset = load_dataset("openwebtext", split="train", streaming=True)

# Defining fraction and calculate number of samples
total_samples = 8000000  # Approximate total samples in OpenWebText
fraction = 0.0001
num_samples = int(total_samples * fraction)
print(f"Using {num_samples} samples")

# Creating a subset
small_dataset = dataset.take(num_samples)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
Using 800 samples


### Load the Tokenizer and Tokenize the Dataset

In [ ]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Defining tokenization function
def tokenize_function(examples):
    # Add padding token to the tokenizer
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(
        examples["text"],
        truncation=True, # Explicitly enable truncation
        padding="max_length",
        max_length=512,
    )

# Applying tokenization to the reduced dataset
tokenized_dataset = small_dataset.map(tokenize_function, batched=True)


In [ ]:
# Defining data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal language modeling (no Masked Language Modeling)
)

### Load the GPT-2 Model

In [ ]:
#Loading model
model = GPT2LMHeadModel.from_pretrained("gpt2")


### Set up Training Parameters

In [ ]:
training_args = TrainingArguments(
   output_dir="fine_tuned_gpt2_openwebtext",
    per_device_train_batch_size=2,
    max_steps=1000,
    save_steps=500,  #Saving checkpoint every 500 steps
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True,  #Enabling mixed precision training
    gradient_accumulation_steps=4,  #Accumulating gradients over 4 steps
)

### Fine-Tune GPT-2

In [ ]:
# Setting up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()



<ipython-input-17-b87b627b92b2>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.191900
200,3.005300
300,2.896800
400,2.810400
500,2.739500
600,2.681500
700,2.631900
800,2.593800
900,2.563200
1000,2.548200


TrainOutput(global_step=1000, training_loss=2.7662266235351565, metrics={'train_runtime': 615.4989, 'train_samples_per_second': 12.998, 'train_steps_per_second': 1.625, 'total_flos': 2090336256000000.0, 'train_loss': 2.7662266235351565, 'epoch': 9.1})

### Save the Fine-Tuned Model

In [ ]:
model.save_pretrained("./fine_tuned_gpt2_openwebtext")
tokenizer.save_pretrained("./fine_tuned_gpt2_openwebtext")

print("Fine-tuning complete. Model saved successfully!")

Fine-tuning complete. Model saved successfully!


### Test the Fine-Tuned Model by generating text

In [ ]:
#Loading the fine-tuned model and tokenizer from Google Drive
finetuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2_openwebtext")
finetuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2_openwebtext")

#Function to generate text
def generate_text(prompt, max_length=100):
    inputs = finetuned_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = finetuned_model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True, #Enabling sampling for temperature and top_p to work
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.2,  #Helping to reduce repetitive outputs
    )
    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

#Testing fine-tuned model with different prompts
test_prompts = [
    "The future of artificial intelligence is",
    "Once upon a time,",
    "The latest advancements in machine learning have",
    "The meaning of love is",
    "In a world where honesty rules,"
]
for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    print(f"Generated Text: {generate_text(prompt)}\n")




Prompt: The future of artificial intelligence is
Generated Text: The future of artificial intelligence is in the air, and with each passing year more researchers work to harness it. But what if we could control how intelligent machines learn?
“It would be cool for computers to have a human head that knows when they're going to hit walls or move their arms around them but not just as awesome on humans who don't know this stuff so far back into our brains—but AI isn't even close yet," says Christopher Herlihy-Mec

Prompt: Once upon a time,
Generated Text: Once upon a time, when the American people were still living in fear of their own government and fearful that it was going to be overthrown by an outside force who might destroy them as well as us all—during those darkest days before our times came for some reason or another we are confronted with this sobering fact: That on March 7th 1947 just three months after America's independence from Great Britain, two black men murdered five Afr